In [141]:
import keras
import numpy as np
import pandas as pd
import tensorflow as tf
import plotly.graph_objects as go
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.preprocessing.sequence import TimeseriesGenerator

In [142]:
filename = "Case_PRB_Europe_Only_true_Steven_Testing_Berat told me to_V0.17.csv"
df = pd.read_csv(filename)

look_back = 4
print(df.info())
df = df.groupby("location")
df = df.get_group("United Kingdom")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10498 entries, 0 to 10497
Columns: 188 entries, Unnamed: 0 to day_int.1
dtypes: float64(15), int64(169), object(4)
memory usage: 15.1+ MB
None


In [143]:
df["date"] = pd.to_datetime(df["date"], infer_datetime_format=True)
df.set_axis(df["date"], inplace = True)

In [144]:
death_data = df["total_deaths"].values
death_data = death_data.reshape((-1, 1))

split_percent = 0.80
split = int(split_percent*len(death_data))

death_train = death_data[:split]
death_test = death_data[split:]

date_train = df["date"][:split]
date_test = df["date"][split:]

# print(death_data)
print(len(death_train))
print(len(death_test))

229
58


In [145]:
train_generator = TimeseriesGenerator(death_train, death_train, length=look_back, batch_size=20)
test_generator = TimeseriesGenerator(death_test, death_test, length=look_back, batch_size=1)

In [146]:
model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(look_back,1)))
model.add(Dense(1))
model.compile(optimizer = "adam", loss="mse", metrics = ["accuracy"])

model.fit_generator(train_generator, epochs=25, verbose=1)


<ipython-input-146-7c83d7131c61>:20: UserWarning:

`Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.



Epoch 1/25
11/11 [==============================] - 4s 16ms/step - loss: 53642.5430 - accuracy: 0.2570
Epoch 2/25
11/11 [==============================] - 0s 16ms/step - loss: 55473.6680 - accuracy: 0.2570
Epoch 3/25
11/11 [==============================] - 0s 16ms/step - loss: 28173.5801 - accuracy: 0.2570
Epoch 4/25
11/11 [==============================] - 0s 16ms/step - loss: 33264.6367 - accuracy: 0.2570
Epoch 5/25
11/11 [==============================] - 0s 16ms/step - loss: 32624.1348 - accuracy: 0.2570
Epoch 6/25
11/11 [==============================] - 0s 16ms/step - loss: 28548.6777 - accuracy: 0.2570
Epoch 7/25
11/11 [==============================] - 0s 16ms/step - loss: 29282.3457 - accuracy: 0.2570
Epoch 8/25
11/11 [==============================] - 0s 16ms/step - loss: 24880.0762 - accuracy: 0.2570
Epoch 9/25
11/11 [==============================] - 0s 17ms/step - loss: 27268.5449 - accuracy: 0.2570
Epoch 10/25
11/11 [==============================] - 0s 17ms/step - loss:

In [147]:
prediction = model.predict(test_generator)

death_train = death_train.reshape((-1))
death_test = death_test.reshape((-1))
prediction = prediction.reshape((-1))

# print(death_train)

# trace1 = go.Scatter(
#     x = date_train,
#     y = death_train,
#     mode = 'lines',
#     name = 'Data'
# )

trace1 = go.Scatter(
    x = date_train,
    y = death_train,
    mode = "lines",
    name = "Data"
)
trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = "lines",
    name = "Prediction"
)
trace3 = go.Scatter(
    x = date_test,
    y = death_test,
    mode="lines",
    name = "Ground Truth"
)
layout = go.Layout(
    title = "Google Stock",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()